# Benchmark an Endpoint

Welcome to this tutorial on benchmarking an endpoint deployed on a SambaNova dedicated node!

If you don't already have an endpoint deployed, please follow one of the workflows described in the [README](./README.md) to deploy an endpoint before proceeding with this tutorial.

Also, please install the benchmarking requirements in the Python kernel used in this Jupyter notebook

`! pip install -r ../../benchmarking/requirements.txt`

## 1.  Imports

In [1]:
import sys
sys.version

'3.11.11 (main, Dec 11 2024, 10:28:39) [Clang 14.0.6 ]'

In [2]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))
import json
import os
import pprint
import getpass
import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
benchmarking_dir = "../../benchmarking/"
sys.path.append(benchmarking_dir + "benchmarking_scripts")
sys.path.append(benchmarking_dir + "prompts")

from synthetic_performance_eval_script import *

## 2. Get endpoint info
To benchmark the endpoint, we will need to obtain some of its information. Note that this information can be obtained from your SambaNova representative.

#### Enter the endpoint url
Run the cell below and then enter the endpoint URL. This should be in the format of `https://my.env/v1/<endpoint_id>/chat/completions` 

In [4]:
endpoint_url = input().strip()
if endpoint_url[-1] == "/":
    endpoint_url = endpoint_url[:-1]
os.environ["SAMBASTUDIO_URL"] = endpoint_url
endpoint_id = endpoint_url.split('/')[-3]
print("Benchmarking Endpoint:", endpoint_id)

Benchmarking Endpoint: ad64853e-c96d-45c8-b98f-72a6a650e73b


#### Enter the endpoint API key

In [5]:
endpoint_key = getpass.getpass().strip()
if len(endpoint_key) > 0:
    os.environ["SAMBASTUDIO_API_KEY"] = endpoint_key
else:
    print("Please enter a valid key")

## 3. Automatically retrieve model list from endpoint (Optional)
Run this section only if you don't the have list of models on the endpoint. Note that running this section requires you to first follow the set up instructions given in the [README](./README.md)
#### Set up environment connector
The connector connects to the remote dedicated environment using the variables defined below

In [6]:
env_url = '/'.join(endpoint_url.split('/')[:3])
env_url

'https://sjc3-e9.sambanova.net'

In [7]:
print("Enter the env access key")
env_key = getpass.getpass().strip()
if len(env_key) > 0:
    os.environ["SAMBASTUDIO_ACCESS_KEY"] = env_key
else:
    print("Please enter a valid key")

Enter the env access key


In [8]:
env_tenant = "default"

In [9]:
from snsdk import SnSdk
sn_env = SnSdk(host_url=env_url, 
                   access_key=env_key, 
                   tenant_id=env_tenant)

#### Get model names in the endpoint

In [10]:
endpoint_info = sn_env.endpoint_info_by_id(endpoint_id)
endpoint_model_id = endpoint_info['targets'][0]["model"]
model_info = sn_env.model_info(endpoint_model_id, job_type="deploy")
model_constituents = [m["name"] for m in model_info["dependencies"]]
sorted(model_constituents)

['Meta-Llama-3.3-70B-Instruct',
 'Qwen-2.5-72B-SD-Qwen-2.5-0.5B',
 'Salesforce--Llama-xLAM-2-70b-fc-r',
 'Salesforce--Llama-xLAM-2-8b-fc-r']

#### Get target model names in the endpoint
Target model names generally differ from model names when the model is a speculative decoding pair

In [11]:
target_models = []
for constituent_name in model_constituents:    
    model_name = constituent_name

    # Check for speculative decoding
    constituent_info = sn_env.model_info(constituent_name, job_type="deploy")
    if 'target_model' in constituent_info['config']:
        target_name = constituent_info['config']['target_model']        
        if len(target_name) > 0:
            model_name = target_name
    target_models.append(model_name)
sorted(target_models)

['Meta-Llama-3.3-70B-Instruct',
 'Qwen2.5-72B-Instruct',
 'Salesforce--Llama-xLAM-2-70b-fc-r',
 'Salesforce--Llama-xLAM-2-8b-fc-r']

## 4. Set up Model Configs for Benchmarking
Note that this section only currently supports a fraction of what the Benchmarking Kit is capable of. You may repurpose this section if you would like to benchmark images or run questions per second (qps).

#### Specify target models
If not automatically set in Step 3, set Target models as a python list

In [12]:
target_models = target_models # ["model1", "model2", "model3"]

#### Specify combinatorial inputs for benchmarking

In [ ]:
input_tokens = [4000, 8000, 16000]
output_tokens = [30]
num_requests = [64]
concurrent_requests = [1, 2, 4, 8, 16, 32]

#### Automatically generate configs

In [14]:
model_configs_df = pd.DataFrame(columns=[
                "model_name",
                "input_tokens",
                "output_tokens",
                "num_requests",
                "concurrent_requests"
                ])
counter = 1
for target_model in target_models:
    for input_token in input_tokens:
        for output_token in output_tokens:
            for num_request in num_requests:
                for concurrent_request in concurrent_requests:
                    model_configs_df.loc[counter] = [
                                        target_model, 
                                        input_token,
                                        output_token,
                                        num_request,
                                        concurrent_request
                                    ]
                    counter += 1


#### Confirm model configs and delete any configs that you would rather exclude

In [15]:
model_configs_df

,model_name,input_tokens,output_tokens,num_requests,concurrent_requests
1,Qwen2.5-72B-Instruct,4000,100,64,1
2,Qwen2.5-72B-Instruct,4000,100,64,2
3,Qwen2.5-72B-Instruct,4000,100,64,4
4,Qwen2.5-72B-Instruct,4000,100,64,8
5,Qwen2.5-72B-Instruct,4000,100,64,16
...,...,...,...,...,...
68,Salesforce--Llama-xLAM-2-70b-fc-r,16000,100,64,2
69,Salesforce--Llama-xLAM-2-70b-fc-r,16000,100,64,4
70,Salesforce--Llama-xLAM-2-70b-fc-r,16000,100,64,8
71,Salesforce--Llama-xLAM-2-70b-fc-r,16000,100,64,16


In [16]:
drop_rows = []
model_configs_df.drop(drop_rows, inplace=True)
model_configs_df

,model_name,input_tokens,output_tokens,num_requests,concurrent_requests
1,Qwen2.5-72B-Instruct,4000,100,64,1
2,Qwen2.5-72B-Instruct,4000,100,64,2
3,Qwen2.5-72B-Instruct,4000,100,64,4
4,Qwen2.5-72B-Instruct,4000,100,64,8
5,Qwen2.5-72B-Instruct,4000,100,64,16
...,...,...,...,...,...
68,Salesforce--Llama-xLAM-2-70b-fc-r,16000,100,64,2
69,Salesforce--Llama-xLAM-2-70b-fc-r,16000,100,64,4
70,Salesforce--Llama-xLAM-2-70b-fc-r,16000,100,64,8
71,Salesforce--Llama-xLAM-2-70b-fc-r,16000,100,64,16


## 5. Run Benchmarking
We will run benchmarking with the configs now

#### Name the benchmarking run
Give the run a unique name so that the configs and results can be saved with that name. Please note that the name should be compatible with file system path naming rules.

In [17]:
run_name = "qwen_llama_salesforce_20250519_1"

#### Configure saving options
Saving makes it easy to re-run the model configs by skipping all the above sections

In [ ]:
output_path = f"{benchmarking_dir} data/benchmarking_tracking_tests/"
config = {
    'model_configs_path': f'{output_path}model_configs_{run_name}.csv', # leave this as is
    'llm_api': 'sambastudio', # leave this as is
    'output_files_dir': f'{output_path}logs/output_files', # each run saved here
    'consolidated_results_dir': f'{output_path}consolidated_results', # consolidated xlsx saved here
    'timeout': 3600,
    'time_delay': 0, # between batches of concurrent requests
}

#### Save configs

In [19]:
with open(f"{output_path}config_{run_name}.yaml", "w") as f:
    yaml.dump(config, f, default_flow_style=False)
model_configs_df.to_csv(config["model_configs_path"], index=False)

#### Run configs

In [20]:
run_benchmarking(config=config, benchmarking_dir=benchmarking_dir, run_name=run_name)

2025-05-19 18:28:05,715 [INFO] Running model_name Qwen2.5-72B-Instruct, input_tokens 4000, output_tokens 100, concurrent_requests 1, num_requests 64, multimodal_img_size na


Running Requests: 100%|██████████| 64/64 [00:10<00:00,  6.27it/s]

2025-05-19 18:28:16,727 [ERROR] Error while running model_name Qwen2.5-72B-Instruct,                 input_tokens 4000,                 output_tokens 100,                 num_requests 64,                 concurrent_requests 1,                 qps 0.0,                 qps_distribution constant                 multimodal_img_size na
2025-05-19 18:28:16,728 [ERROR] Unexpected error happened when executing requests:                
- Error while running LLM API requests. Check your model name, LLM API type, env variables and endpoint status.                
Additional messages:
- Error: 'choices' at streamed event: {"error":{"code":null,"message":"Requested generation length 100 is not possible! The provided prompt is 4028 tokens long, so generating 100 tokens requires a sequence length of 4128, but the maximum supported sequence length is just 4096!","param":null,"type":"Error"}}
2025-05-19 18:28:16,728 [INFO] Time delay: 0 seconds
2025-05-19 18:28:16,730 [INFO] Running model_name Qwen2.5

Running Requests: 100%|██████████| 64/64 [00:16<00:00,  4.17it/s]

2025-05-19 18:28:33,831 [ERROR] Error while running model_name Qwen2.5-72B-Instruct,                 input_tokens 4000,                 output_tokens 100,                 num_requests 64,                 concurrent_requests 2,                 qps 0.0,                 qps_distribution constant                 multimodal_img_size na
2025-05-19 18:28:33,832 [ERROR] Unexpected error happened when executing requests:                
- Error while running LLM API requests. Check your model name, LLM API type, env variables and endpoint status.                
Additional messages:
- Error: 'choices' at streamed event: {"error":{"code":null,"message":"Requested generation length 100 is not possible! The provided prompt is 4028 tokens long, so generating 100 tokens requires a sequence length of 4128, but the maximum supported sequence length is just 4096!","param":null,"type":"Error"}}
2025-05-19 18:28:33,834 [INFO] Time delay: 0 seconds
2025-05-19 18:28:33,836 [INFO] Running model_name Qwen2.5

...

2025-05-19 20:28:27,878 [INFO] Tasks Executed!
2025-05-19 20:28:27,879 [INFO] Benchmarking results obtained for model Salesforce--Llama-xLAM-2-70b-fc-r queried with the sambastudio API.
2025-05-19 20:28:27,898 [INFO] Building Client Metrics Summary for metric: client_ttft_s
2025-05-19 20:28:27,900 [INFO]     p5 = 30.6743
2025-05-19 20:28:27,900 [INFO]     p25 = 59.763
2025-05-19 20:28:27,901 [INFO]     p50 = 59.7972
2025-05-19 20:28:27,901 [INFO]     p75 = 59.8309
2025-05-19 20:28:27,902 [INFO]     p90 = 59.8934
2025-05-19 20:28:27,902 [INFO]     p95 = 59.9009
2025-05-19 20:28:27,903 [INFO]     p99 = 59.9292
2025-05-19 20:28:27,903 [INFO]     mean = 56.5063
2025-05-19 20:28:27,904 [INFO]     min = 6.8467
2025-05-19 20:28:27,904 [INFO]     max = 59.94
2025-05-19 20:28:27,905 [INFO]     stddev = 10.7724
2025-05-19 20:28:27,905 [INFO] Building Client Metrics Summary for metric: client_end_to_end_latency_s
2025-05-19 20:28:27,907 [INFO]     p5 = 31.5067
2025-05-19 20:28:27,907 [INFO]     p

Running Requests: 100%|██████████| 64/64 [08:05<00:00,  7.58s/it]

2025-05-19 20:36:33,742 [INFO] Tasks Executed!
2025-05-19 20:36:33,744 [INFO] Benchmarking results obtained for model Salesforce--Llama-xLAM-2-70b-fc-r queried with the sambastudio API.
2025-05-19 20:36:33,764 [INFO] Building Client Metrics Summary for metric: client_ttft_s
2025-05-19 20:36:33,766 [INFO]     p5 = 30.7753
2025-05-19 20:36:33,767 [INFO]     p25 = 120.3658
2025-05-19 20:36:33,767 [INFO]     p50 = 120.4083
2025-05-19 20:36:33,768 [INFO]     p75 = 120.4476
2025-05-19 20:36:33,768 [INFO]     p90 = 120.524
2025-05-19 20:36:33,768 [INFO]     p95 = 120.5507
2025-05-19 20:36:33,769 [INFO]     p99 = 120.5746
2025-05-19 20:36:33,769 [INFO]     mean = 106.2557
2025-05-19 20:36:33,770 [INFO]     min = 6.8724
2025-05-19 20:36:33,770 [INFO]     max = 120.5788
2025-05-19 20:36:33,770 [INFO]     stddev = 30.3777
2025-05-19 20:36:33,771 [INFO] Building Client Metrics Summary for metric: client_end_to_end_latency_s
2025-05-19 20:36:33,772 [INFO]     p5 = 31.5773
2025-05-19 20:36:33,773 [I

Running Requests: 100%|██████████| 64/64 [08:05<00:00,  7.58s/it]

2025-05-19 20:44:43,412 [INFO] Tasks Executed!
2025-05-19 20:44:43,413 [INFO] Benchmarking results obtained for model Salesforce--Llama-xLAM-2-70b-fc-r queried with the sambastudio API.
2025-05-19 20:44:43,465 [INFO] Building Client Metrics Summary for metric: client_ttft_s
2025-05-19 20:44:43,473 [INFO]     p5 = 31.6024
2025-05-19 20:44:43,475 [INFO]     p25 = 127.0769
2025-05-19 20:44:43,476 [INFO]     p50 = 241.5003
2025-05-19 20:44:43,478 [INFO]     p75 = 241.5663
2025-05-19 20:44:43,481 [INFO]     p90 = 241.66
2025-05-19 20:44:43,485 [INFO]     p95 = 241.6718
2025-05-19 20:44:43,489 [INFO]     p99 = 242.0805
2025-05-19 20:44:43,490 [INFO]     mean = 183.3796
2025-05-19 20:44:43,491 [INFO]     min = 7.734
2025-05-19 20:44:43,491 [INFO]     max = 242.6297
2025-05-19 20:44:43,491 [INFO]     stddev = 76.9887
2025-05-19 20:44:43,491 [INFO] Building Client Metrics Summary for metric: client_end_to_end_latency_s
2025-05-19 20:44:43,493 [INFO]     p5 = 32.4285
2025-05-19 20:44:43,493 [INF